In [1]:
# Cell 1：切到项目根目录 + 基础检查
import os

## 切换工作目录到 Aristotle/TRACE 项目根
os.chdir("/workspace")
print("Current working directory:", os.getcwd())

print("Files here:")
print(os.listdir("."))
print("\nLogicNLI data dir:")
print(os.listdir("data/LogicNLI"))

## 方案 1：告诉 wandb 这是脚本，而不是 notebook（最推荐）
## 在你的 notebook 开头加入：
os.environ["WANDB_NOTEBOOK_NAME"] = "baseline_ Aristotle_LogicNLI_v2.ipynb"
os.environ["WANDB_DISABLE_NOTEBOOK"] = "true"


Current working directory: /workspace
Files here:
['.git', '.gitignore', 'README.md', 'aristotle.png', 'data', 'evaluate.py', 'negate.py', 'prompts', 'requirements.txt', 'search_resolve.py', 'translate_decompose.py', 'utils.py', 'experiments', 'notebooks', 'apptainer', 'scripts', 'TRACE.md', '__pycache__', 'results', 'logs', 'wandb']

LogicNLI data dir:
['dev.json']


In [2]:
# Cell 2：安全设置 OpenAI & wandb（不要提交到 Git）
import os
import getpass

# 建议用 getpass，避免 key 明文出现在界面 / 历史中
openai_key = getpass.getpass("OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_key

# 如果需要手动登录 wandb（通常一次登录之后 cookie 会保存）
import wandb

wandb_api = getpass.getpass("wandb API key (留空则跳过 login): ")
if wandb_api.strip():
    os.environ["WANDB_API_KEY"] = wandb_api
    wandb.login(key=wandb_api)
else:
    print("跳过 wandb.login（用已有登录状态）")

OpenAI API key:  ········
wandb API key (留空则跳过 login):  ········


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find baseline_ Aristotle_LogicNLI_v2.ipynb.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/chengjie.zheng001/.netrc
wandb: Currently logged in as: chengjiezheng (chengjiezheng-umb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
# Cell 3：实验配置 & 初始化 wandb run
import time
import wandb
from datetime import datetime
import os

# 基本配置
PROJECT_DIR = "/workspace"
DATASET_NAME = "LogicNLI"
SPLIT = "dev"
MODEL_NAME = "gpt-4o"      # 你也可以换成 gpt-4.1 等
MAX_NEW_TOKENS = 2048
BATCH_NUM = 1

RUN_NAME = f"{DATASET_NAME}-{SPLIT}-{MODEL_NAME}-{int(time.time())}"

config = {
    "dataset": DATASET_NAME,
    "split": SPLIT,
    "model_name": MODEL_NAME,
    "max_new_tokens": MAX_NEW_TOKENS,
    "batch_num": BATCH_NUM,
    "pipeline": "Aristotle_baseline",
}

run = wandb.init(
    project="TRACE_LogicNLI",  # 你可以换成自己的项目名
    name=RUN_NAME,
    config=config,
    reinit=True,
)

print("wandb run url:", run.get_url())

### 在你原来的 Cell 3（实验配置 & wandb init） 末尾，加上几行：
## 全局日志缓存（把每一步 stdout/stderr 收集到这里）
LOG_BUFFERS = []

## 准备 logs 目录 & 日志文件名（数据集 + split + 模型 + 时间戳）
os.makedirs("logs", exist_ok=True)
LOG_FILENAME = f"logs/{DATASET_NAME}_{SPLIT}_{MODEL_NAME}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
print("Log will be saved to:", LOG_FILENAME)

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


wandb run url: https://wandb.ai/chengjiezheng-umb/TRACE_LogicNLI/runs/a0y834ma
Log will be saved to: logs/LogicNLI_dev_gpt-4o_20251202_060846.txt


In [13]:
# Cell 4：封装一个运行命令的小工具（方便打印 & 报错）(Version 1)
import subprocess
import textwrap

def run_cmd(cmd: str):
    """在 /workspace 下执行 shell 命令，打印 stdout/stderr，并在出错时抛异常。"""
    print("\n" + "="*80)
    print(">>> Running command:")
    print(textwrap.indent(cmd, prefix="    "))
    print("="*80 + "\n")

    result = subprocess.run(
        cmd,
        shell=True,
        text=True,
        capture_output=True,
        cwd=PROJECT_DIR,
    )

    if result.stdout:
        print("STDOUT:\n" + result.stdout)
    if result.stderr:
        print("\nSTDERR:\n" + result.stderr)

    # 如果 returncode 非 0，抛出异常，方便 debug
    result.check_returncode()
    return result.stdout


In [14]:
# Cell 4：封装一个运行命令的小工具（方便打印 & 报错）(Version 2)
# 修改 run_cmd：把每次命令的输出都写进 LOG_BUFFERS

import subprocess
import textwrap

def run_cmd(cmd: str, tag: str = ""):
    """在 /workspace 下执行 shell 命令，并把 stdout/stderr 记录到全局 LOG_BUFFERS。"""
    global LOG_BUFFERS

    header = f"[{tag}] " if tag else ""
    print("\n" + "="*80)
    print(f">>> {header}Running command:")
    print(textwrap.indent(cmd, prefix="    "))
    print("="*80 + "\n")

    result = subprocess.run(
        cmd,
        shell=True,
        text=True,
        capture_output=True,
        cwd=PROJECT_DIR,
    )

    stdout = result.stdout or ""
    stderr = result.stderr or ""

    if stdout:
        print("STDOUT:\n" + stdout)
    if stderr:
        print("\nSTDERR:\n" + stderr)

    # 🔹 把本次命令的所有信息 append 到 LOG_BUFFERS
    LOG_BUFFERS.append(
        "\n".join([
            "="*80,
            f"[{header}CMD]",
            cmd,
            "-"*80,
            "[STDOUT]",
            stdout,
            "-"*80,
            "[STDERR]",
            stderr,
            "="*80,
            "",
        ])
    )

    # 出错直接抛异常，方便你在 Notebook 中看到
    result.check_returncode()
    return stdout

In [5]:
# Cell 4：封装一个运行命令的小工具（方便打印 & 报错）(Version 3)
import subprocess
import textwrap

def run_cmd(cmd: str, tag: str = ""):
    """在 /workspace 下执行 shell 命令，并把 stdout/stderr 记录到 LOG_BUFFERS 和日志文件."""
    global LOG_BUFFERS, LOG_FILENAME

    header = f"[{tag}] " if tag else ""
    print("\n" + "="*80)
    print(f">>> {header}Running command:")
    print(textwrap.indent(cmd, prefix="    "))
    print("="*80 + "\n")

    result = subprocess.run(
        cmd,
        shell=True,
        text=True,
        capture_output=True,   # 命令跑完后一次性拿到输出
        cwd=PROJECT_DIR,
    )

    stdout = result.stdout or ""
    stderr = result.stderr or ""

    if stdout:
        print("STDOUT:\n" + stdout)
    if stderr:
        print("\nSTDERR:\n" + stderr)

    # 🔹 组织一段完整的日志块
    log_block = "\n".join([
        "="*80,
        f"[{header}CMD]",
        cmd,
        "-"*80,
        "[STDOUT]",
        stdout,
        "-"*80,
        "[STDERR]",
        stderr,
        "="*80,
        "",
    ])

    # 1）存到内存里的 LOG_BUFFERS
    LOG_BUFFERS.append(log_block)

    # 2）立刻追加写入到 txt 文件（如果文件不存在会自动创建）
    with open(LOG_FILENAME, "a") as f:
        f.write(log_block)

    # 出错直接抛异常
    result.check_returncode()
    return stdout


In [ ]:
# Cell 5：Step A – translate_decompose（LogicNLI-dev）
cmd_translate = f"""
python translate_decompose.py \
  --api_key "$OPENAI_API_KEY" \
  --model_name "{MODEL_NAME}" \
  --data_path "./data" \
  --dataset_name "{DATASET_NAME}" \
  --split {SPLIT} \
  --max_new_tokens {MAX_NEW_TOKENS} \
  --batch_num {BATCH_NUM}
"""

start_time = time.time()
### out_translate = run_cmd(cmd_translate)
out_translate = run_cmd(cmd_translate, tag="TRANSLATE_DECOMPOSE")
wandb.log({"time_translate_sec": time.time() - start_time})



>>> [TRANSLATE_DECOMPOSE] Running command:

    python translate_decompose.py   --api_key "$OPENAI_API_KEY"   --model_name "gpt-4o"   --data_path "./data"   --dataset_name "LogicNLI"   --split dev   --max_new_tokens 2048   --batch_num 1




In [7]:
# Cell 6：Step B – negate（生成 negated_data）
cmd_negate = f"""
python negate.py \
  --dataset_name "{DATASET_NAME}" \
  --model "{MODEL_NAME}"
"""

start_time = time.time()
### out_negate = run_cmd(cmd_negate)
out_negate = run_cmd(cmd_negate, tag="NEGATE")
wandb.log({"time_negate_sec": time.time() - start_time})


>>> [NEGATE] Running command:

    python negate.py   --dataset_name "LogicNLI"   --model "gpt-4o"




In [9]:
# Cell 7：Step C1 – search_resolve（no negation）
cmd_search_no_neg = f"""
python search_resolve.py \
  --api_key "$OPENAI_API_KEY" \
  --model_name "{MODEL_NAME}" \
  --data_path "./data" \
  --dataset_name "{DATASET_NAME}" \
  --split {SPLIT} \
  --negation False \
  --max_new_tokens {MAX_NEW_TOKENS} \
  --batch_num {BATCH_NUM}
"""

start_time = time.time()
### out_search_no_neg = run_cmd(cmd_search_no_neg)
out_search_no_neg = run_cmd(cmd_search_no_neg, tag="SEARCH_RESOLVE_NO_NEG")
wandb.log({"time_search_no_neg_sec": time.time() - start_time})


In [ ]:
# Cell 8：Step C2 – search_resolve（with negation）
cmd_search_neg = f"""
python search_resolve.py \
  --api_key "$OPENAI_API_KEY" \
  --model_name "{MODEL_NAME}" \
  --data_path "./data" \
  --dataset_name "{DATASET_NAME}" \
  --split {SPLIT} \
  --negation True \
  --max_new_tokens {MAX_NEW_TOKENS} \
  --batch_num {BATCH_NUM}
"""

start_time = time.time()
### out_search_neg = run_cmd(cmd_search_neg)
out_search_neg = run_cmd(cmd_search_neg, tag="SEARCH_RESOLVE_NEG")
wandb.log({"time_search_neg_sec": time.time() - start_time})


In [ ]:
# Cell 9：Step D – evaluate + 解析 accuracy + 记录到 wandb
import re

cmd_eval = f"""
python evaluate.py \
  --dataset_name "{DATASET_NAME}" \
  --model "{MODEL_NAME}"
"""

start_time = time.time()
### out_eval = run_cmd(cmd_eval)
out_eval = run_cmd(cmd_eval, tag="EVALUATE")
elapsed_eval = time.time() - start_time
wandb.log({"time_evaluate_sec": elapsed_eval})

# 从 stdout 中解析 Accuracy
match = re.search(r"Accuracy:\s*([0-9.]+)", out_eval)
if match:
    acc = float(match.group(1))
    print(f"\nParsed Accuracy = {acc:.4f}")
    wandb.log({"dev_accuracy": acc})
else:
    print("\n⚠ 没能从 evaluate 输出中解析出 Accuracy，请检查 evaluate.py 的打印格式。")


In [ ]:
# Cell 10：收尾（wandb.finish）(version 1)
wandb.finish()
print("Done. Run URL:", run.get_url())


In [ ]:
# Cell 10：收尾（wandb.finish）(version 2)
# 先把所有日志写入 txt
full_log_text = "\n".join(LOG_BUFFERS)

with open(LOG_FILENAME, "w") as f:
    f.write(full_log_text)

print("✅ Log saved to:", LOG_FILENAME)

# 可选：把 log 文件同步到 wandb，方便以后从 web 下载
try:
    wandb.save(LOG_FILENAME)
    print("Log file also saved to wandb.")
except Exception as e:
    print("wandb.save failed:", e)

# 结束 wandb run
wandb.finish()
print("Done. Run URL:", run.get_url())


In [ ]:
# Cell 10：收尾（wandb.finish）(version 3)
# 可选：把 log 文件同步到 wandb
try:
    wandb.save(LOG_FILENAME)
    print("Log file also saved to wandb.")
except Exception as e:
    print("wandb.save failed:", e)

wandb.finish()
print("Done. Run URL:", run.get_url())
print("Final log file:", LOG_FILENAME)
